In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("IMDB_movies2.csv")
df

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0004134,Hypocrites,Hypocrites,1915,1/20/1915,Drama,54,USA,English,Lois Weber,Lois Weber,Hobart Bosworth Productions,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",The parallel stories of a modern preacher and ...,6.6,429,NaN,NaN,NaN,NaN,41,8.0
1,tt0004707,Tillie's Punctured Romance,Tillie's Punctured Romance,1914,12/21/1914,Comedy,82,USA,English,"Mack Sennett, Charles Bennett",NaN,Keystone Film Company,"Marie Dressler, Charles Chaplin, Mabel Normand...",A con man from the city dupes a wealthy countr...,6.3,3164,"$50,000",NaN,NaN,NaN,37,18.0
2,tt0005078,The Cheat,The Cheat,1915,12/13/1915,"Drama, Romance",59,USA,English,Cecil B. DeMille,"Hector Turnbull, Jeanie Macpherson",Jesse L. Lasky Feature Play Company,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...","A venal, spoiled stockbroker's wife impulsivel...",6.6,2217,"$17,311",NaN,NaN,NaN,38,19.0
3,tt0006333,"20,000 Leagues Under the Sea","20,000 Leagues Under the Sea",1916,12/24/1916,"Action, Adventure, Sci-Fi",105,USA,English,Stuart Paton,Jules Verne,Williamson Submarine Film Corporation,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",A French professor and his daughter accompany ...,6.2,1440,"$200,000",NaN,NaN,NaN,26,16.0
4,tt0009611,Shoulder Arms,Shoulder Arms,1918,10/27/1918,"Comedy, War",45,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charlie is a boot camp private who has a dream...,7.3,5411,NaN,NaN,NaN,NaN,35,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14062,tt9426186,Revenger,Revenger,2018,12/6/2018,"Action, Crime, Thriller",101,South Korea,"Korean, English",Seung-Won Lee,NaN,Green Fish Pictures,"Bruce Khan, Hee-soon Park, Jin-Seo Yoon",A former police investigator decides to go to ...,5.7,2658,NaN,NaN,NaN,NaN,58,20.0
14063,tt9495224,Black Mirror: Bandersnatch,Black Mirror: Bandersnatch,2018,12/28/2018,"Drama, Mystery, Sci-Fi",90,"USA, UK",English,David Slade,Charlie Brooker,House Of Tomorrow,"Fionn Whitehead, Craig Parkinson, Alice Lowe, ...",A young programmer starts to question reality ...,7.2,99095,NaN,NaN,NaN,NaN,950,90.0
14064,tt9664078,American Son,American Son,2019,11/1/2019,Drama,90,USA,English,Kenny Leon,Christopher Demos-Brown,Netflix,"Kerry Washington, Steven Pasquale, Jeremy Jord...",An estranged couple reunite in a Florida polic...,5.7,2930,NaN,NaN,NaN,33.0,205,16.0
14065,tt9675464,Gang Leader,Gang Leader,2019,9/13/2019,"Action, Comedy",155,India,"English, Telugu",Vikram K. Kumar,"Vikram K. Kumar, Mukund Pandey",Mythri Movie Makers,"Nani, Kartikeya Gummakonda, Priyanka Arulmohan...",Nani plays the gang leader of five women belon...,7.7,1318,NaN,NaN,"$161,321",NaN,30,3.0


If you visualize the dataset, you will see that it has many extra info about a movie. We don’t need all of them. So, we choose keywords, cast, genres and director column to use as our feature set(the so called “content” of the movie).

In [2]:
df = df[["title", "year", "genre", "country", "director", "writer", "production_company", "actors", "description", "avg_vote"]]
df.head()

,title,year,genre,country,director,writer,production_company,actors,description,avg_vote
0,Hypocrites,1915,Drama,USA,Lois Weber,Lois Weber,Hobart Bosworth Productions,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",The parallel stories of a modern preacher and ...,6.6
1,Tillie's Punctured Romance,1914,Comedy,USA,"Mack Sennett, Charles Bennett",NaN,Keystone Film Company,"Marie Dressler, Charles Chaplin, Mabel Normand...",A con man from the city dupes a wealthy countr...,6.3
2,The Cheat,1915,"Drama, Romance",USA,Cecil B. DeMille,"Hector Turnbull, Jeanie Macpherson",Jesse L. Lasky Feature Play Company,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...","A venal, spoiled stockbroker's wife impulsivel...",6.6
3,"20,000 Leagues Under the Sea",1916,"Action, Adventure, Sci-Fi",USA,Stuart Paton,Jules Verne,Williamson Submarine Film Corporation,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",A French professor and his daughter accompany ...,6.2
4,Shoulder Arms,1918,"Comedy, War",USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charlie is a boot camp private who has a dream...,7.3


In [3]:
df['avg_vote'] = df['avg_vote'].astype(str)
df['year'] = df['year'].astype(str)
df['description'] = df['description'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [4]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.append('A')
stop.append('The')
df['keywords'] = df['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,title,year,genre,country,director,writer,production_company,actors,description,avg_vote,keywords
0,Hypocrites,1915,Drama,USA,Lois Weber,Lois Weber,Hobart Bosworth Productions,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",The parallel stories of a modern preacher and ...,6.6,parallel stories modern preacher medieval monk...
1,Tillie's Punctured Romance,1914,Comedy,USA,"Mack Sennett, Charles Bennett",NaN,Keystone Film Company,"Marie Dressler, Charles Chaplin, Mabel Normand...",A con man from the city dupes a wealthy countr...,6.3,con man city dupes wealthy country girl marriage.
2,The Cheat,1915,"Drama, Romance",USA,Cecil B. DeMille,"Hector Turnbull, Jeanie Macpherson",Jesse L. Lasky Feature Play Company,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...","A venal, spoiled stockbroker's wife impulsivel...",6.6,"venal, spoiled stockbroker's wife impulsively ..."
3,"20,000 Leagues Under the Sea",1916,"Action, Adventure, Sci-Fi",USA,Stuart Paton,Jules Verne,Williamson Submarine Film Corporation,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",A French professor and his daughter accompany ...,6.2,French professor daughter accompany Captain Ne...
4,Shoulder Arms,1918,"Comedy, War",USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charlie is a boot camp private who has a dream...,7.3,Charlie boot camp private dream hero goes dari...
...,...,...,...,...,...,...,...,...,...,...,...
14062,Revenger,2018,"Action, Crime, Thriller",South Korea,Seung-Won Lee,NaN,Green Fish Pictures,"Bruce Khan, Hee-soon Park, Jin-Seo Yoon",A former police investigator decides to go to ...,5.7,former police investigator decides go prison i...
14063,Black Mirror: Bandersnatch,2018,"Drama, Mystery, Sci-Fi","USA, UK",David Slade,Charlie Brooker,House Of Tomorrow,"Fionn Whitehead, Craig Parkinson, Alice Lowe, ...",A young programmer starts to question reality ...,7.2,young programmer starts question reality adapt...
14064,American Son,2019,Drama,USA,Kenny Leon,Christopher Demos-Brown,Netflix,"Kerry Washington, Steven Pasquale, Jeremy Jord...",An estranged couple reunite in a Florida polic...,5.7,An estranged couple reunite Florida police sta...
14065,Gang Leader,2019,"Action, Comedy",India,Vikram K. Kumar,"Vikram K. Kumar, Mukund Pandey",Mythri Movie Makers,"Nani, Kartikeya Gummakonda, Priyanka Arulmohan...",Nani plays the gang leader of five women belon...,7.7,Nani plays gang leader five women belonging di...


In [5]:
df.dtypes

title                 object
year                  object
genre                 object
country               object
director              object
writer                object
production_company    object
actors                object
description           object
avg_vote              object
keywords              object
dtype: object

In [6]:
features = ['director', 'year', 'genre', 'keywords', 'production_company']

Our next task is to create a function for combining the values of these columns into a single string.

In [7]:
def combine_features(row):
    return row['director']+" "+row['year']+" "+row['genre']+" "+row['keywords']+" "+row['production_company']

Now, we need to call this function over each row of our dataframe. But, before doing that, we need to clean and preprocess the data for our use. We will fill all the NaN values with blank string in the dataframe.

In [8]:
for feature in features:
    df[feature] = df[feature].fillna('') #filling all NaNs with blank string

df["combined_features"] = df.apply(combine_features,axis=1) #applying combined_features() method over each rows of dataframe and storing the combined string in "combined_features" column


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
df.iloc[0].combined_features

'Lois Weber 1915 Drama parallel stories modern preacher medieval monk, Gabriel Ascetic, killed ignorant mob making nude statue representing Truth, also represented ghostly naked girl flits throughout film. Hobart Bosworth Productions'

Now that we have obtained the combined strings, we can now feed these strings to a CountVectorizer() object for getting the count matrix.

In [10]:
cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(df["combined_features"]) #feeding combined strings(movie contents) to CountVectorizer() object
count_matrix

<14067x29655 sparse matrix of type '<class 'numpy.int64'>'
	with 328533 stored elements in Compressed Sparse Row format>

At this point, 60% work is done. Now, we need to obtain the cosine similarity matrix from the count matrix.

In [11]:
cosine_sim = cosine_similarity(count_matrix)

Now, we will define two helper functions to get movie title from movie index and vice-versa.

In [12]:
df = df.reset_index()
df.head()

,index,title,year,genre,country,director,writer,production_company,actors,description,avg_vote,keywords,combined_features
0,0,Hypocrites,1915,Drama,USA,Lois Weber,Lois Weber,Hobart Bosworth Productions,"Courtenay Foote, Myrtle Stedman, Herbert Stand...",The parallel stories of a modern preacher and ...,6.6,parallel stories modern preacher medieval monk...,Lois Weber 1915 Drama parallel stories modern ...
1,1,Tillie's Punctured Romance,1914,Comedy,USA,"Mack Sennett, Charles Bennett",NaN,Keystone Film Company,"Marie Dressler, Charles Chaplin, Mabel Normand...",A con man from the city dupes a wealthy countr...,6.3,con man city dupes wealthy country girl marriage.,"Mack Sennett, Charles Bennett 1914 Comedy con ..."
2,2,The Cheat,1915,"Drama, Romance",USA,Cecil B. DeMille,"Hector Turnbull, Jeanie Macpherson",Jesse L. Lasky Feature Play Company,"Fannie Ward, Sessue Hayakawa, Jack Dean, James...","A venal, spoiled stockbroker's wife impulsivel...",6.6,"venal, spoiled stockbroker's wife impulsively ...","Cecil B. DeMille 1915 Drama, Romance venal, sp..."
3,3,"20,000 Leagues Under the Sea",1916,"Action, Adventure, Sci-Fi",USA,Stuart Paton,Jules Verne,Williamson Submarine Film Corporation,"Dan Hanlon, Edna Pendleton, Curtis Benton, All...",A French professor and his daughter accompany ...,6.2,French professor daughter accompany Captain Ne...,"Stuart Paton 1916 Action, Adventure, Sci-Fi Fr..."
4,4,Shoulder Arms,1918,"Comedy, War",USA,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Edna Purviance, Charles Chaplin, Syd Chaplin, ...",Charlie is a boot camp private who has a dream...,7.3,Charlie boot camp private dream hero goes dari...,"Charles Chaplin 1918 Comedy, War Charlie boot ..."


In [13]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

Our next step is to get the title of the movie that the user currently likes. Then we will find the index of that movie. After that, we will access the row corresponding to this movie in the similarity matrix. Thus, we will get the similarity scores of all other movies from the current movie. Then we will enumerate through all the similarity scores of that movie to make a tuple of movie index and similarity score. This will convert a row of similarity scores like this- `[1 0.5 0.2 0.9]` to this- `[(0, 1) (1, 0.5) (2, 0.2) (3, 0.9)]` . Here, each item is in this form- (movie index, similarity score).

In [35]:
movie_user_likes = "Bedknobs and Broomsticks"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index])) #accessing the row corresponding to given movie to find all the similarity scores for that movie and then enumerating over it


Now comes the most vital point. We will sort the list `similar_movies` according to similarity scores in descending order. Since the most similar movie to a given movie will be itself, we will discard the first element after sorting the movies.

In [36]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

Now, we will run a loop to print first 5 entries from `sorted_similar_movies` list.

In [37]:
i=0
print("Top 5 similar movies to "+movie_user_likes+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

Top 5 similar movies to Bedknobs and Broomsticks are:

Pete's Dragon
Robin Hood
The Rescuers
The Love Bug
The Island at the Top of the World
Mary Poppins
